# Bankrupt dataset ETL

Librerie varie da installare

In [ ]:
#!pip install pandas
#!pip install matplotlib
#!pip install seaborn

Inclusione delle librerie utilizzate

In [ ]:
import pandas as pd
import os
import glob
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Variabili di gestione files

In [ ]:
# Path of the directory containing the bankruptcy companies .csv files, can be changed
PATH_BANKRUPT = r"C:\Users\Andre\OneDrive - Università degli Studi di Parma\Tirocinio\Dataset\Bankruptcy_rielaborati_G"

# True = export summary file in the OUTPUT_PATH
to_export = False

# True = handle Nan and inf values of indexes as 0 / False = drop them
handle_non_numeric_values = False

# Path of the desired output file, can be changed
OUTPUT_PATH = r"C:\Users\Andre\OneDrive - Università degli Studi di Parma\Tirocinio\Dataset_output"

Leggo, per esempio, i file .csv contenuti in una directory e le sue sottodirectories (BANKRUPT)

In [ ]:
# Dataset with all the csv files
input_dataset = pd.DataFrame()

# Path of each csv file
# Path descrption: Bankruptcy_rielaborate_G/*.csv
csv_files = glob.glob(os.path.join(PATH_BANKRUPT, "**/*.csv"), recursive=True)

# Loop over the list of csv files
for f in csv_files:

    # Print the location and filename
    print('Location:', f)
    print('File Name:', f.split("\\")[-1])

    # Read each csv file
    partial_dataset = pd.read_csv(f, sep=";", dtype=object, thousands=",", decimal=".")

    # Append the partial dataset
    input_dataset = pd.concat([input_dataset, partial_dataset])




print("Dimensione dataset delle società in attivo: ", input_dataset.shape)

In [ ]:
input_dataset

Gestisco i nomi delle colonne in modo consistente

In [ ]:
for col in input_dataset.columns:
    old_col_name = col
    new_col_name = re.sub("\r\nEUR\r\nYear - \d|\r\nYear - \d|\nEUR\nYear - \d|Year - \d|EUR", "", old_col_name)
    new_col_name = re.sub("\r\n|\r|\n", " ", new_col_name)
    input_dataset.rename(columns={old_col_name: new_col_name}, inplace=True)

In [ ]:
# Combine all columns with the same name
input_dataset = input_dataset.groupby(level=0, axis=1).sum()

Seleziono solo le colonne che mi interessano per il calcolo degli indici

In [ ]:
# Massive amount of columns needed to compute different financial estimators
to_keep_columns = ['Company name',
                   'Province',
                   'Employees',
                   'Accounting closing date Last avail. yr',
                   'Legal form',
                   'Legal status',
                   'Total financial charges',
                   "TOTAL SHAREHOLDERS' FUNDS",
                   'TOTAL CURRENT ASSETS',
                   'OPERATING MARGIN',
                   "Total depreciation, amortization and writedowns",
                   'Provisions fo risks and charges',
                   'Other provisions',
                   'Tax payable',
                   'Tax payable beyond 12 months',
                   'Due to social security institutions',
                   'Due to social security institutions - beyond 12 months',
                   'TOTAL FIXED ASSETS',
                   'PROFIT (LOSS) GROUP',
                   'Trade accounts',
                   'Trade accounts - beyond 12 months',
                   'Due to suppliers',
                   'Due to suppliers - beyond 12 months',
                   'TOTAL INVENTORIES',
                   'Due to banks',
                   'Due to banks - beyond 12 months',
                   'Due to other lenders',
                   'Due to other lenders - beyond 12 months',
                   'Bonds',
                   'Bonds beyond 12 months',
                   'Due to shar. for loans',
                   'Due to sharesholders for loans - beyond 12 months',
                   'Revenues from sales and services',
                   'TOTAL LIQUID FUNDS',
                   'TOTAL PAYABLES',
                   'Total payables during period',
                   'Total payables after period',
                   'TOTAL ASSETS',
                   "TOTAL LIABILITIES AND SHAREHOLDERS' FUNDS",
                   'Raw, consum. mat. and goods for resale',
                   'Services']

input_dataset = input_dataset[to_keep_columns]

In [ ]:
input_dataset.head()

Rinomino certe colonne per essere consistente con il dataset delle companie in attivo

In [ ]:
map_columns = {
    "Company name": "Ragione sociale",
    "Employees": "Number of employees",
    "Legal form": "Legal Form",
    "Legal status": "Legal Status",
    "TOTAL SHAREHOLDERS' FUNDS": "TOTAL SHAREHOLDERS FUNDS",
    "Accounting closing date Last avail. yr": "Accounting closing date",
    "TOTAL ASSETS": "Total asset",
    "TOTAL LIABILITIES AND SHAREHOLDERS' FUNDS": "TOTAL LIABILITIES AND SHAREHOLDERS FUNDS",
    "Due to shar. for loans": "Due to shareholders for loans"
}
input_dataset.rename(columns=map_columns, inplace=True)

In [ ]:
input_dataset

Tutti i valori non disponibili sono rimpiazzati con NaN

In [ ]:
input_dataset.replace(to_replace="n.a.", value=np.NaN, inplace=True)
input_dataset.replace(to_replace="n.d.", value=np.NaN, inplace=True)
input_dataset.replace(to_replace="#VALUE!", value=np.NaN, inplace=True)
input_dataset.replace(to_replace="#NOME?", value=np.NaN, inplace=True)
input_dataset.replace(to_replace="UDATACHI", value=np.NaN, inplace=True)
input_dataset.isna().sum()

Rimuovo tutti i record contenenti valori NaN

In [ ]:
input_dataset.dropna(inplace=True)
input_dataset.isna().sum()

In [ ]:
print("Dimensione dataset filtrato dai valori mancanti e con numero di colonne ridotte: ", input_dataset.shape)

Cast delle colonne del dataset a tipi numerici e descrittivi (stringhe)

In [ ]:
str_cols = ['Ragione sociale',
            'Province',
            'Legal Form',
            'Legal Status',
            'Accounting closing date']

for col in input_dataset.columns:
    if col in str_cols:
        input_dataset[col] = input_dataset[col].astype(str)
    else:
        input_dataset[col] = pd.to_numeric(input_dataset[col])

Rendo la colonna Legal Status contenente un unico valore (Bankruptcy)

In [ ]:
input_dataset["Legal Status"] = "Bankruptcy"

Breve panoramica del dataset bankrupt

In [ ]:
input_dataset.describe().T

Inizio a calcolare i vari indicatori

In [ ]:
# Max estimator value
MAX_VALUE_INDEX = 10
# Min estimator value
MIN_VALUE_INDEX = -10

Indicatori di allerta

In [ ]:
input_dataset["OF/Ricavi"] = input_dataset["Total financial charges"] / input_dataset["Revenues from sales and services"]

input_dataset["PN/Totale Debiti"] = input_dataset["TOTAL SHAREHOLDERS FUNDS"] / input_dataset["TOTAL PAYABLES"]

input_dataset["Att. Br/Pass. Br"] = input_dataset["TOTAL CURRENT ASSETS"] / input_dataset["Total payables during period"]

input_dataset["EBITDA/Attivo"] = (input_dataset["OPERATING MARGIN"] +
                                  input_dataset["Total depreciation, amortization and writedowns"] +
                                  input_dataset["Provisions fo risks and charges"] +
                                  input_dataset["Other provisions"]) / input_dataset["Total asset"]

input_dataset["Deb. Prev + Trib/Attivo"] = (input_dataset["Tax payable"] +
                                            input_dataset["Tax payable beyond 12 months"] +
                                            input_dataset["Due to social security institutions"] +
                                            input_dataset["Due to social security institutions - beyond 12 months"]) / input_dataset["Total asset"]

In [ ]:
allerta_indexes = ["OF/Ricavi", "PN/Totale Debiti", "Att. Br/Pass. Br", "EBITDA/Attivo", "Deb. Prev + Trib/Attivo"]
input_dataset[allerta_indexes]

In [ ]:
pd.set_option('display.float_format', lambda x: '%0.4f' % x)
input_dataset[allerta_indexes].describe().T

Indicatori provenienti dalla letteratura

In [ ]:
input_dataset["Attivo fisso/Totale Passivita"] = input_dataset["TOTAL FIXED ASSETS"] / input_dataset["TOTAL LIABILITIES AND SHAREHOLDERS FUNDS"]

input_dataset["ROE"] = input_dataset["PROFIT (LOSS) GROUP"] / input_dataset["TOTAL SHAREHOLDERS FUNDS"]

input_dataset["Tempo medio riscossione (TMR)"] = (input_dataset["Trade accounts"] +
                                                  input_dataset["Trade accounts - beyond 12 months"] ) / (input_dataset["Revenues from sales and services"]/360)

input_dataset["Tempo medio di pagamento (TMP)"] = (input_dataset["Due to suppliers"] +
                                                   input_dataset["Due to suppliers - beyond 12 months"]) / (
                                                          (input_dataset["Raw, consum. mat. and goods for resale"] +
                                                           input_dataset["Services"])/360)

input_dataset["Indice liquidita Primaria (Acid Test)"] = (input_dataset["TOTAL CURRENT ASSETS"] -
                                                          input_dataset["TOTAL INVENTORIES"]) / input_dataset["Total payables during period"]

input_dataset["Indice di copertura delle immob."] = input_dataset["TOTAL FIXED ASSETS"] / (input_dataset["TOTAL SHAREHOLDERS FUNDS"] + input_dataset["Total payables after period"])

input_dataset["Indice copertura On.Fin."] = input_dataset["OPERATING MARGIN"] / input_dataset["Total financial charges"]

input_dataset["Indice copertura On.Fin. Con liquidita"] = (input_dataset["OPERATING MARGIN"] +
                                                           input_dataset["Total depreciation, amortization and writedowns"] +
                                                           input_dataset["Provisions fo risks and charges"] +
                                                           input_dataset["Other provisions"]) / input_dataset["Total financial charges"]
# Common numerator - begin
numerator = (input_dataset["Due to banks"] +
             input_dataset["Due to banks - beyond 12 months"] +
             input_dataset["Due to other lenders"] +
             input_dataset["Due to other lenders - beyond 12 months"] +
             input_dataset["Bonds"] +
             input_dataset["Bonds beyond 12 months"] +
             input_dataset["Due to shareholders for loans"] +
             input_dataset["Due to sharesholders for loans - beyond 12 months"] -
             input_dataset["TOTAL LIQUID FUNDS"])

input_dataset["PFN/EBITDA"] = numerator / (input_dataset["OPERATING MARGIN"] +
                                           input_dataset["Total depreciation, amortization and writedowns"] +
                                           input_dataset["Provisions fo risks and charges"] +
                                           input_dataset["Other provisions"])

input_dataset["PFN/PN"] = numerator / input_dataset["TOTAL SHAREHOLDERS FUNDS"]

input_dataset["PFN/Vendite"] = numerator / input_dataset["Revenues from sales and services"]

input_dataset["Gearing"] = numerator / (numerator + input_dataset["TOTAL SHAREHOLDERS FUNDS"])
# Common numerator - end

input_dataset["ROS"] = input_dataset["OPERATING MARGIN"] / input_dataset["Revenues from sales and services"]

input_dataset["Turnover capitale investito"] = (input_dataset["Revenues from sales and services"]/360) / input_dataset["Total asset"]

input_dataset["Working capital/net sales"] = input_dataset["TOTAL CURRENT ASSETS"] / input_dataset["Revenues from sales and services"]

input_dataset["Cash/Current Liabilities"] = input_dataset["TOTAL LIQUID FUNDS"] / input_dataset["Total payables during period"]

input_dataset["Accounts receivable/inventory"] = (input_dataset["Trade accounts"] + input_dataset["Trade accounts - beyond 12 months"]) / input_dataset["TOTAL INVENTORIES"]

input_dataset["EBIT/interest expenses"] = input_dataset["OPERATING MARGIN"] / input_dataset["Total financial charges"]

In [ ]:
letteratura_indexes = ["Attivo fisso/Totale Passivita",
                       "ROE",
                       "Tempo medio riscossione (TMR)",
                       "Tempo medio di pagamento (TMP)",
                       "Indice liquidita Primaria (Acid Test)",
                       "Indice di copertura delle immob.",
                       "Indice copertura On.Fin.",
                       "Indice copertura On.Fin. Con liquidita",
                       "PFN/EBITDA",
                       "PFN/PN",
                       "PFN/Vendite",
                       "Gearing",
                       "ROS",
                       "Turnover capitale investito",
                       "Working capital/net sales",
                       "Cash/Current Liabilities",
                       "Accounts receivable/inventory",
                       "EBIT/interest expenses"]
input_dataset[letteratura_indexes]

In [ ]:
input_dataset[letteratura_indexes].describe().T

Indicatori di Altman Z-score

In [ ]:
input_dataset["Att.Br/Attivo"] = input_dataset["TOTAL CURRENT ASSETS"] / input_dataset["Total asset"]

input_dataset["Utili non Distr./Attivo"] = input_dataset["PROFIT (LOSS) GROUP"] / input_dataset["Total asset"]

input_dataset["ROI"] = input_dataset["OPERATING MARGIN"] / input_dataset["Total asset"]

input_dataset["Ricavi/Attivo"] = input_dataset["Revenues from sales and services"] / input_dataset["Total asset"]

input_dataset["EBITDA/Totale Debiti"] = (input_dataset["OPERATING MARGIN"] +
                                         input_dataset["Total depreciation, amortization and writedowns"] +
                                         input_dataset["Provisions fo risks and charges"] +
                                         input_dataset["Other provisions"]) / input_dataset["TOTAL PAYABLES"]

In [ ]:
altman_indexes = ["Att.Br/Attivo", "Utili non Distr./Attivo", "ROI", "Ricavi/Attivo", "EBITDA/Totale Debiti"]
input_dataset[altman_indexes]

In [ ]:
input_dataset[altman_indexes].describe().T

Rimuovo tutti i record con valori NaN o inf

In [ ]:
if handle_non_numeric_values:
    filtered_dataset = input_dataset.replace(to_replace=[np.inf, -np.inf, np.NaN], value=0)
else:
    filtered_dataset = input_dataset.replace(to_replace=[np.inf, -np.inf], value=np.NaN).dropna()

Guardo correlazione indicatori

In [ ]:
indexes_corr = filtered_dataset[allerta_indexes + letteratura_indexes + altman_indexes].corr()

sns.set_theme(style="white")

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(indexes_corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(30, 30))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(indexes_corr, mask=mask, cmap=cmap, vmax=1,
            center=0, annot=True, fmt='.2f',
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

Ci sono indici estremamente correlati fra di loro, rimuovo alcuni di essi

In [ ]:
removed_indexes = [# Found from active
                    "EBITDA/Attivo",
                    "OF/Ricavi",
                    "Att. Br/Pass. Br",
                    "Attivo fisso/Totale Passivita",
                    "Indice liquidita Primaria (Acid Test)",
                    "Indice copertura On.Fin.",
                    "Indice copertura On.Fin. Con liquidita",
                    "PFN/Vendite",
                    "Turnover capitale investito",
                    "Utili non Distr./Attivo",
                    # Found from bankrupt
                    "ROE",
                    "ROI",
                    "Indice di copertura delle immob."]
filtered_dataset.drop(columns=removed_indexes, axis=1, inplace=True)

In [ ]:
indexes = allerta_indexes + letteratura_indexes + altman_indexes

for index in removed_indexes:
    indexes.remove(index)

indexes_corr = filtered_dataset[indexes].corr()

sns.set_theme(style="white")

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(indexes_corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(20, 20))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(indexes_corr, mask=mask, cmap=cmap, vmax=1,
            center=0, annot=True, fmt='.2f',
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [ ]:
filtered_dataset[indexes].describe().T

Esporto in csv e pickle

In [ ]:
if to_export:
    if handle_non_numeric_values:
        filtered_dataset.to_csv(OUTPUT_PATH + "/bankruptcy_out_big.csv")
        filtered_dataset.to_pickle(OUTPUT_PATH + "/bankruptcy_out_big.pkl")
    else:
        filtered_dataset.to_csv(OUTPUT_PATH + "/bankruptcy_out_small.csv")
        filtered_dataset.to_pickle(OUTPUT_PATH + "/bankruptcy_out_small.pkl")